In [246]:
from os import path
from scipy.stats import kde
import numpy as np
import pandas as pd
from bokeh.io import show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.plotting import figure
from bokeh.palettes import Dark2
output_notebook()

Loading BokehJS ...

In [247]:
tweets = pd.read_csv('../data/feat_tweets_201804181405.csv')
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125964 entries, 0 to 125963
Data columns (total 15 columns):
user             125964 non-null object
retweet_count    125964 non-null float64
source           125964 non-null object
text             125964 non-null object
is_reply         125964 non-null int64
is_retweet       125964 non-null int64
city             125964 non-null object
utc_time         125964 non-null object
local_time       125964 non-null object
weekday          125964 non-null int64
month            125964 non-null int64
words            119417 non-null object
num_words        125964 non-null int64
badwords         7848 non-null object
num_badwords     125964 non-null int64
dtypes: float64(1), int64(6), object(8)
memory usage: 14.4+ MB


In [248]:
data = tweets[tweets['is_retweet'] == 0].reset_index()
# data['badwords'] = data['badwords'].fillna('').str.split(';')
data['has_badword'] = data['num_badwords'] > 0
data.head()

,index,user,retweet_count,source,text,is_reply,is_retweet,city,utc_time,local_time,weekday,month,words,num_words,badwords,num_badwords,has_badword
0,0,UID0000000000269739792,0.0,Twitter for Android,@bruce_arthur I am worried about the Bucks in ...,1,0,toronto,2018-04-03 23:59:59+00:00,2018-04-03 19:59:59-04:00,1,4,i;am;worried;about;the;bucks;in;the;first;roun...,21,NaN,0,False
1,2,UID0000000004161881183,0.0,Twitter for iPhone,Thanks for organizing this! 👏🏻👌🏻 https://t.co/...,0,0,toronto,2018-04-03 23:59:59+00:00,2018-04-03 19:59:59-04:00,1,4,thanks;for;organizing;this,4,NaN,0,False
2,5,UID0000000004101243735,0.0,Twitter for iPhone,@sasha_cresswell I’m 10000% down to go after w...,1,0,toronto,2018-04-03 23:59:59+00:00,2018-04-03 19:59:59-04:00,1,4,i;m;1000;down;to;go;after;work,8,NaN,0,False
3,8,UID0747950508564971520,0.0,Twitter for iPad,"Thanks for the fun chat tonight, another great...",0,0,toronto,2018-04-03 23:59:58+00:00,2018-04-03 19:59:58-04:00,1,4,thanks;for;the;fun;chat;tonight;another;great;...,10,NaN,0,False
4,12,UID0000000000183698387,0.0,Twitter for iPhone,"Sleep, art, sleep or art... why cant i sleep a...",0,0,toronto,2018-04-03 23:59:58+00:00,2018-04-03 19:59:58-04:00,1,4,sleep;art;sleep;or;art;why;cant;i;sleep;and;ar...,16,NaN,0,False


In [273]:
def agg_feat(dat, size=5000):
    _dat = dat.iloc[np.random.randint(0, dat.shape[0], size)]
    return pd.Series({
        'n_twt': size,
        'n_wrd': sum(_dat['num_words']),
        'prp_wrd': sum(_dat['num_badwords'])/sum(_dat['num_words']),
        'prp_twt': sum(_dat['has_badword'])/_dat.shape[0],
        'bad_words': ';'.join(_dat['badwords'].dropna())
    })

In [275]:
data.groupby(['city']).apply(agg_feat).reset_index()

,city,bad_words,n_twt,n_wrd,prp_twt,prp_wrd
0,chicago,dick;shit;ass;dick;hell;shit;ass;hell;fucking;...,5000,47667,0.0760,0.009042
1,los_angeles,bullshit;damn;bitch;bullshit;bullshit;fuck;shi...,5000,47486,0.0532,0.006886
2,new_york,damn;fuck;fuck;bitch;cum;shit;damn;porn;fucked...,5000,48642,0.0536,0.006229
3,toronto,ass;poop;hell;hell;shit;shit;fucking;fuck;hell...,5000,50737,0.0374,0.003942
4,vancouver,shit;fuck;fucking;jism;ass;ass;fucking;sex;que...,5000,54628,0.0324,0.003588


In [287]:
badwords = ';'.join(data['badwords'].dropna()).split(';')
u_badwords, n_badwords = np.unique(badwords, return_counts=True)
tot_n_badwords = len(badwords)
prp_badword = {}
for ind, wrd in enumerate(u_badwords):
    prp_badword.update({wrd: n_badwords[ind]/tot_n_badwords})
prp_badword

{'anal': 0.0017476406850751485,
 'anus': 0.0003495281370150297,
 'arsehole': 0.0003495281370150297,
 'ass': 0.08109052778748689,
 'asses': 0.0013981125480601187,
 'asshat': 0.0003495281370150297,
 'asshole': 0.005592450192240475,
 'assholes': 0.002446696959105208,
 'balls': 0.008388675288360714,
 'bastard': 0.0010485844110450892,
 'bitch': 0.049982523593149246,
 'bitchass': 0.0003495281370150297,
 'bitches': 0.013981125480601188,
 'bitching': 0.0003495281370150297,
 'bitchy': 0.0010485844110450892,
 'blow job': 0.0003495281370150297,
 'blowjob': 0.0006990562740300594,
 'boner': 0.0003495281370150297,
 'boob': 0.0010485844110450892,
 'boobs': 0.004893393918210416,
 'bullshit': 0.008738203425375743,
 'bum': 0.0013981125480601187,
 'butt': 0.007689619014330654,
 'chode': 0.0003495281370150297,
 'cock': 0.005941978329255505,
 'cocks': 0.0013981125480601187,
 'cocksucker': 0.0003495281370150297,
 'coon': 0.0003495281370150297,
 'cox': 0.0013981125480601187,
 'crap': 0.009786787836420832,
 '

In [255]:
agg['bad_words'].str.split(';').apply(len)

0    158
1     80
2     81
3    111
4    134
Name: bad_words, dtype: int64

In [256]:
data.shape

(49450, 17)

In [228]:
def bootstrap(dat):
    boot = {}
    for i in range(1000):
        _data = dat.iloc[np.random.randint(0, dat.shape[0], 10000)]
        boot.update({ i:{
            'prop_wrd': sum(_data['num_badwords'])/sum(_data['num_words']),
            'prop_twt': sum(_data['has_badword'])/_data.shape[0],
            'bad_words': ';'.join(_data['badwords'].dropna())
        }})
    return pd.DataFrame.from_dict(boot, 'index')
    
boot = data.groupby('city').apply(bootstrap).reset_index()

In [229]:
boot[['city', 'prop_wrd', 'prop_twt']].groupby('city').agg([np.mean, np.std])

prop_wrd            prop_twt          
                 mean       std      mean       std
city                                               
chicago      0.008738  0.000334  0.073318  0.002673
los_angeles  0.007094  0.000317  0.059169  0.002485
new_york     0.005949  0.000290  0.049522  0.002172
toronto      0.004481  0.000227  0.041377  0.002013
vancouver    0.003876  0.000220  0.036191  0.001844

In [245]:
plt = figure(
    title='Foul city',
    output_backend='webgl',
    tools=['box_zoom'],
    plot_width=600,
    plot_height=400
)
for ind, (city, _dat) in enumerate(boot.groupby(['city'])):
    hist, edges = np.histogram(_dat['prop_twt'], density=True, bins=50)
    plt.quad(
        top=hist, bottom=0, left=edges[:-1], right=edges[1:],
        fill_color=Dark2[5][ind], line_color=None, fill_alpha=0.3, legend=city.replace('_', ' ').title())
show(plt)

In [236]:
Dark2[5]

['#1b9e77', '#d95f02', '#7570b3', '#e7298a', '#66a61e']

In [202]:
agg

,city,bad_words,num_twt,num_wrd,prop_twt,prop_wrd
0,chicago,bitch;ass;hell;ass;lmfao;fuck;lmfao;ass;shit;n...,10496,101642,0.073361,0.008746
1,los_angeles,sex;shitfaced;shit;shit;fucking;damn;lmfao;ass...,6801,65445,0.059109,0.007090
2,new_york,shitty;bitch;shit;shitty;prick;shitbag;cunt;sh...,7753,75585,0.049658,0.005967
3,toronto,damn;hell;damn;butt;damn;nigga;fucking;ass;bum...,10987,111437,0.041413,0.004487
4,vancouver,shitty;shit;shitty;lmfao;damn;shit;fucking;ass...,13413,147456,0.036159,0.003872


In [51]:
source = ColumnDataSource(res)
plt = figure(
    title='Foul city',
    x_range=list(res['city']),
    output_backend='webgl',
    tools=['box_zoom'],
    plot_width=600,
    plot_height=400
)
plt.grid.visible = False
plt.yaxis.visible = False
plt.outline_line_color = None
plt.toolbar_location = None
plt.vbar(x='city', top='freq', width=0.8, source=source)
# plt.ann
show(plt);